In [2]:
import azureml.core
from azureml.core import Workspace, Datastore
import pandas as pd

# set up workspace
ws= Workspace.from_config() 

# Take a look at Workspace
ws.get_details()

# set up datastores
dstore = ws.get_default_datastore()

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Default datastore name'] = dstore.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T


,
SDK version,1.13.0
Subscription ID,33125c98-8730-4ada-8519-4282d89758eb
Workspace,mme-test-sa
Resource Group,sample
Location,westus
Default datastore name,workspaceblobstore


In [3]:
from azureml.core.compute import AmlCompute, ComputeTarget

# Choose a name for your cluster.
amlcompute_cluster_name = "cpucluster"

found = False
# Check if this compute target already exists in the workspace.
cts = ws.compute_targets
if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
    found = True
    print('Found existing compute target.')
    compute = cts[amlcompute_cluster_name]
    
if not found:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D13_V2',
                                                           min_nodes=3,
                                                           max_nodes=20)
    # Create the cluster.
    compute = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)
    
print('Checking cluster status...')
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
compute.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)

# For a more detailed view of current AmlCompute status, use get_status().

Found existing compute target.
Checking cluster status...
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.core import Experiment

experiment = Experiment(ws, 'manymodels-forecasting-pipeline')

In [5]:
from azureml.core import Datastore

dstore = ws.get_default_datastore()

from azureml.core.dataset import Dataset

#filedst_10_models = Dataset.get_by_name(ws, name='oj_data_small_inference')
#filedst_10_models_input = filedst_10_models.as_named_input('forecast_10_models')

# filedst_all_models = Dataset.get_by_name(ws, name='oj_data_inference')
# filedst_all_models_input = filedst_all_models.as_named_input('forecast_all_models')

Energy50 = Dataset.get_by_name(ws, name = 'energy50_train')
Energy50_input = Energy50.as_named_input('Energy50')


In [6]:
##If the environment is registered

from azureml.core import Environment

forecast_env = Environment.get(ws,"many_models_environment_automl")

In [17]:
from azureml.core import Model

tags_dict = {}
tags_dict['RunId'] = "f404b0f1-6644-409c-b890-83cc5a61c362"
tags_dict['Hash'] = "08b144b0b87c3983288c93ab01da35534d99c6f899c970473516d7027e9ab706"


# automl_8a4e66d75f855ad38a5e71b17992cacacdf6843cedb02484a45eb7872aec4f39
# automl_8a4e66d75f855ad38a5e71b17992cacacdf6843cedb02484a45eb7872aec4f39
# automl_1cf1878b6e29d68a457c61ce56cbf9ee1cf72d5d76e04ff1006808aab82165b4
# automl_08b144b0b87c3983288c93ab01da35534d99c6f899c970473516d7027e9ab706

model_list = Model.list(ws,name="automl_08b144b0b87c3983288c93ab01da35534d99c6f899c970473516d7027e9ab706",
                        tags=tags_dict, latest=True)
print(model_list)

[]
